# News Sentiment Demo
Also scratch space 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
import numpy as np 
import pandas as pd 
import re 

from util import *

## EDA 

In [3]:
sample_path = os.path.join('data/archive', 'df_1920.csv')
sample_data = pd.read_csv(sample_path, index_col=0)
sample_data.head()

,year,sentence
0,1920,"ROME, Jan. 2.--It is of the utmost importance ..."
1,1920,FIND GIRL AND DOG DEAD.; Candy Near Bodies in...
2,1920,There was opened yesterday in the Art Departme...
3,1920,Free Champagne at the Vanderbilt.
4,1920,"20 Seized at Portland, Ore."


In [14]:
# count_words_by_year 
count_words_by_year(1920)

new             20787
will            19520
yesterday       12641
today           12063
no              10298
                ...  
gubillier           1
odium               1
inclinations        1
infectcd            1
othrwise            1
Length: 70407, dtype: int64

In [15]:
# document_frequency 
document_frequency(1920, 'yesterday')

0.09131404785434073

In [9]:
document_frequency(1920, 'today')

0.06955050459677178

We see right away although yesterday and today do not differ much in total occurrence, they have huge difference in the proportion of appearance in each news report. 